# Condensation of water from humid air around a droplet

Description

    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 06/11/2020
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [15]:
# Constants
k_air = lambda T: 7.940e-5*T + 2.340e-3 # W/m-K in air [Incropera et al]
Cpa = lambda T: 2.667e-6*(T**3) - 1.8e-3*(T**2) + 4.233e-1*T + 9.72e2 # kJ/kg-K for air [Incropera et al]
rho_a = lambda T: 3.5783e2 * T**(-1.0047)  # kg/m3 for air [Incropera et al]

kv = lambda T: 8.4569e-5*T - 1.1409e-2 # W/m-K in R-134a vapor [Laesecke et al 1992]
Cpv = lambda T: 102.03*(1.94006e1 + 2.58531e-1*T + -1.29665e-4*T**2) # J/kg-K for R-134a vapor [Dupont]
Cpl = lambda T: 5.2419e-3*(T**2) - 1.6133*T + 8.6750e2 # J/kg-K for R-134a saturated liquid at 1 atm [Dupont]

rho_w = lambda T: 1.5733e-4*(T**3) - 1.4079e-1*(T**2) + 4.1748e1*T - 3.1054e3 # kg/m3 for subcooled water [Hare & Sorensen]
Cpw = lambda T: -1.1930e-2*(T**3) + 1.0037e1*(T**2) - 2.8151e3*T + 2.6740e5 # kJ/kg-K for water [Angell 1982]
hfg = lambda T: 2500.8 -2.36*(T-273.15) + 0.0016*(T-273.15)**2 - 0.00006*(T-273.15)**3 # kJ/kg for water [Rogers & Yau 1989]

D = 2.82e-5 # m^2/s  (water in air)

In [51]:
# Antoine Equation for vapor pressure
#    T [K]
#    RH [0-1]
def Pv(T,RH):
    return 1e5*RH*np.exp(4.6543-(1435.264/(T-64.848)))

In [67]:
# Magnus dew point temperature
#    T [K]
#    RH [0-1]
def Tdew(T,RH):
    return 273.15 + (257.14*np.log(Pv(T,RH)/611.21)/(18.678-np.log(Pv(T,RH)/611.21)))

In [24]:
# Model function for radius of vapor field around the droplets
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
#     rho_m = mixture density [kg/m^3]
#     Nd = number density [m^-3]
#     rho_v = vapor density [kg/m^3]
def frv(x,y,rho_m,Nd,rho_v):
    f1 = 3*x*(1-y)*rho_m
    f2 = 4*Nd*np.pi*rho_v
    return (f1/f2)**0.33333

In [25]:
# Function for vapor-air heat transfer
#     q = heat transfer [W]
#     T = liquid-vapor equilibrium temperature [K]
#     Tair = air temperature [K]
#     Nd = number density [m^-3]
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
#     rho_v = vapor density [kg/m^3]
#     rho_m = mixture density [kg/m^3
def fheat(q,T,Tair,Nd,d,x,y,rho_v,rho_m):
    f1 = 4*np.pi*k_air(Tair)*(T-Tair)*Nd
    f2 = rho_v*((2./d) - (1./frv(x,y,rho_m,Nd,rho_v)))
    return (f1/f2)-q

In [40]:
# Function for vapor-liquid rate change of enthalpy
#     q = heat transfer [W]
#     dTdt = dT/dt, rate change of liquid-vapor temperature [K/s]
#     T = liquid-vapor equilibrium temperature [K]
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
def fhlv(q,dTdt,x,y,T):
    return (x*Cpv(T)+(1-x)*Cpl(T))*(1-y)*dTdt

In [28]:
# Function for water mass transport rate using Fick's Law
#     D = Diffusion coefficient [m^2/s]
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
#     rho_m = mixture density [kg/m^3]
#     Nd = number density [m^-3]
#     rho_v = vapor density [kg/m^3]
#     Tair = air temperature [K]
#     RH = Relative humidity [0-1]
#     V = cell volume [m^3]
#     p = absolute pressure [Pa]
def dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p):
    r_v=frv(x,y,rho_m,Nd,rho_v)
    f1 = 4*D*np.pi*(r_v**2)*Pv(Tair,RH)*rho_w(Tair)*y*rho_v*Nd*V
    f2 = 3*p*rho_a(Tair)
    return f1/f2

In [73]:
# Function for air and water vapor rate change of enthalpy
#     q = heat transfer [W]
#     dTadt = dTair/dt, rate change of air temperature [K/s]
#     rho_m = mixture density [kg/m^3
#     Nd = number density [m^-3]
#     Tair = air temperature [K]
#     mc = mass of condensed water [kg]
#     RH = Relative humidity [0-1]
#     p = absolute pressure [Pa]
def fhaw(q,dTadt,rho_m,V,Tair,mc,RH,p):
    dmc_dt = dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p)
    f1 = y*Cpa(Tair)*dTadt
    f2 = (Cpw(Tair)/(rho_m*V))*dmc_dt*(Tair-Tdew(Tair,RH))
    f3 = (Cpw(Tair)/(rho_m*V))*mc*dTadt
    f4 = hfg(Tair)*dmc_dt/(rho_m*V)
    return f1 + f2 + f3 + f4 - q

In [95]:
# Constants from CFD.
T=250
Nd=1e9
d=5e-6
x=1e-3
y=.5
rho_v=1.2
rho_m=200
RH=.95
V=1e-9
p=101325


# Initial guesses
q=0
Tair=280
dTdt=(Tair-T)
dTadt=-dTdt
mc=0

print(dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p)) # rate of water transport [kg/s]
print(fheat(q,T,Tair,Nd,d,x,y,rho_v,rho_m)*1e-3) # vapor-air heat transfer [kW]
print(fhlv(q,dTdt,x,y,T)*1e-3) # enthalpy change in the liquid-vapor mixture [kW]
print(fhaw(q,dTadt,rho_m,V,Tair,mc,RH,p)*1e-3) # enthalpy change in the moist air [kW]

5.217678549939228e-10
-19.478504424396824
11.981235321804375
-15.581555671049898
